In [7]:
from runner import evaluate_performance

In [8]:
"""
accuracy_score() computes the number of well predicted / number of all examples
evaluate_performance() function evaluates the performance of decision trees, random forests and logistic regression,
average over 1,000 trials and 
returns a 3x3 matrix stats that contains mean accuracies and standard deviation of accuracies(written in brackets)
"""
stats = evaluate_performance()
print ("Decision Tree Accuracy = ", stats[0, 0], " (", stats[0, 1], ")")
print ("Random Forest Tree Accuracy = ", stats[1, 0], " (", stats[1, 1], ")")
print ("Logistic Reg. Accuracy = ", stats[2, 0], " (", stats[2, 1], ")")

Decision Tree Accuracy =  0.74776119403  ( 0.0454917929974 )
Random Forest Tree Accuracy =  0.805970149254  ( 0.044776119403 )
Logistic Reg. Accuracy =  0.798507462687  ( 0.039063439786 )
